In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re
from utils import TJData, setup_env, APIHandler, nan_to_int
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api.models import Comarca, Serventia, Assunto, Classe, ClasseAssunto, Competencia, Processo, ProcessoUnico, TipoPersonagem
# objeto para interagir com a minha API
api = APIHandler('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

In [2]:
TJData.csv

{'assunto': 'Parameters/ASSUNTO.csv',
 'assunto_2': 'Parametros/DCP_ASSUNTO.csv.gz',
 'assunto_3': 'ASSUNTO.csv',
 'tipo_pedido_mp': 'Parameters/DCP_DCP_TIPOPEDIDOMP.csv',
 'tipo_vinculo': 'Parameters/DCP_TIPOVINCULO.CSV',
 'serventias_full': 'Parameters/serventias_full.csv',
 'ato_juiz': 'Parameters/ATOJUIZ.csv',
 'ato_juiz_2': 'Parametros/DCP_ATOJUIZ.csv.gz',
 'destinatario': 'Parameters/DCP_DESTINATARIO.CSV',
 'tipo_vinculo_processo': 'Parameters/DCP_TIPOVINCULOPROCESSO.csv',
 'tipo_andamento': 'Parameters/tipoandamento.csv',
 'classe': 'Parameters/cLASSE.csv',
 'orgao_mp': 'Parameters/DCP_ORGAOMP.CSV',
 'grupo_processamento': 'Parameters/GRUPOPROCESSAMENTO.csv',
 'tipo_ato_juiz': 'Parameters/TIPOATOJUIZ.csv',
 'comarca': 'Parameters/DCP_Comarca.csv',
 'tipo_categoria_serventia': 'Parameters/DCP_TIPOCATEGORIASERVENTIA.csv',
 'tipo_destinatario': 'Parameters/proccorp_tipodestinatario.csv',
 'competencia': 'Parameters/DCP_Competencia.csv',
 'tipo_remessa': 'Parameters/DCP_TIPOREMESSA.